In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd

C:\Users\ylou2\.conda\envs\myproject\lib\site-packages\pydantic\_internal\_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
C:\Users\ylou2\.conda\envs\myproject\lib\site-packages\pydantic\_internal\_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
C:\Users\ylou2\.conda\envs\myproject\lib\site-packages\pydantic\_internal\_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


# TO DO change community name and monthly fixed charge

In [2]:
community = 'columbia' # TODO change community name
monthly_fixed_charge = 13 # TODO change monthly fixed charge

In [3]:
my_run = BuildStockQuery(db_name='euss-final',
                        table_name='euss_res_final_2018_550k_20220901',
                        workgroup='factsheets',
                        buildstock_type='resstock',
                        skip_reports=True)

INFO:buildstock_query.query_core:Loading euss_res_final_2018_550k_20220901 ...
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [4]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
community_results_bill = pd.DataFrame()
all_frame = []

# Electricity TOU bill

Get building samples and building ids

In [5]:
upgrade_name_list = ['baseline',
                     'Basic Enclosure',
                     'Enhanced Enclosure',
                     'Mininum Efficiency Heat Pump with Electric Heat Backup',
                     'High Efficiency Heat Pump with Electric Heat Backup',
                     'Mininum Efficiency Heat Pump with Existing Heat Backup',
                     'Basic Enclosure + HPWH + High Efficiency HP/Electric Backup',
                     'Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup',
                     'Heat Pump Water Heater',
                     'Electric Clothes Dryer',
                     'Heat Pump Clothes Dryer',
                     'Electric Cooking',
                     'Induction Cooking',
                     'Mininum Efficiency Whole Home Electrification',
                     'High Efficiency Whole Home Electrification',
                     'Basic Enclosure + High Efficiency Whole Home Electrification',
                     'Enhanced Enclosure + High Efficiency Whole Home Electrification'] # 17 in total
results = {}
for up in range (17):
    results[f'{up}'] = community_results.loc[community_results['upgrade_name'] == upgrade_name_list[up]]
    
building_ids_list = {}
for up in results:
    building_ids_list[f'{up}'] = results[up].building_id.tolist()

get euss TOU bill

In [6]:
def euss_TOU_bill(upgrade_id, building_ids_list):
    euss = my_run.utility.calculate_tou_bill(
        meter_col = ['fuel_use__electricity__total__kwh',
                     'end_use__electricity__clothes_dryer__kwh',
                     'end_use__electricity__range_oven__kwh'],
        rate_map = (f"data_/community_cost/TOU_{community}_weekday_cost.csv",
                    f"data_/community_cost/TOU_{community}_weekend_cost.csv"),
        upgrade_id = upgrade_id,
        group_by = ["building_id"],
        restrict = [(my_run.ts_bldgid_column, building_ids_list)],
        collapse_ts = True,
    )
    
    euss['cbill.electricity__TOU__usd'] = euss['fuel_use__electricity__total__kwh__TOU__dollars']/euss['units_count'] + monthly_fixed_charge * 12
    euss['cbill.electricity__clothes_dryer__TOU__usd'] = euss['end_use__electricity__clothes_dryer__kwh__TOU__dollars']/euss['units_count']
    euss['cbill.electricity__range_oven__TOU__usd'] = euss['end_use__electricity__range_oven__kwh__TOU__dollars']/euss['units_count']
    euss = euss.drop(['sample_count', 'units_count', 
                      'fuel_use__electricity__total__kwh__TOU__dollars',
                      'end_use__electricity__clothes_dryer__kwh__TOU__dollars',
                      'end_use__electricity__range_oven__kwh__TOU__dollars'], axis=1)
    
    return euss

INFO:buildstock_query.aggregate_query:Aggregation done accross timestamps. Result no longer a timeseries.
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.
INFO:buildstock_query.aggregate_query:Aggregation done accross timestamps. Result no longer a timeseries.
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 1.
INFO:buildstock_query.aggregate_query:Aggregation done accross timestamps. Result no longer a timeseries.
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 2.
INFO:buildstock_query.aggregate_query:Aggregation done accross timestamps. Result no longer a timeseries.
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 3.
INFO:buildstock_query.aggregate_query:Aggregation done accross timestamps. Result no longer a timeseries.
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 4.
INFO:buildstock_query.aggregate_query:Aggregation done accross timestamps. Result no longer a timeseries.
INFO:buildstoc

In [ ]:
euss00 = euss_TOU_bill('0', building_ids_list['0'])
euss00 = euss00.rename(columns={'cbill.electricity__TOU__usd': 'baseline_cbill.electricity__TOU__usd',
                       'cbill.electricity__clothes_dryer__TOU__usd': 'baseline_cbill.electricity__clothes_dryer__TOU__usd',
                       'cbill.electricity__range_oven__TOU__usd': 'baseline_cbill.electricity__range_oven__TOU__usd'})
euss01 = euss_TOU_bill('1', building_ids_list['1'])
euss02 = euss_TOU_bill('2', building_ids_list['2'])
euss03 = euss_TOU_bill('3', building_ids_list['3'])
euss04 = euss_TOU_bill('4', building_ids_list['4'])
euss05 = euss_TOU_bill('5', building_ids_list['5'])
euss06 = euss_TOU_bill('6', building_ids_list['8'])
euss07 = euss_TOU_bill('7', building_ids_list['13'])
euss08 = euss_TOU_bill('8', building_ids_list['14'])
euss09 = euss_TOU_bill('9', building_ids_list['15'])
euss10 = euss_TOU_bill('10', building_ids_list['16'])

Baseline

In [7]:
up00 = results['0'].join(euss00.set_index('building_id'), on = 'building_id')
all_frame.append(up00)

Package 1. Basic Enclosure - upgrade 1

In [8]:
up01_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['1'])]
up01 = results['1'].join(up01_baseline.set_index('building_id'), on='building_id')
up01 = up01.join(euss01.set_index('building_id'), on='building_id')
up01['saving_cbill.electricity_TOU_usd'] = up01['baseline_cbill.electricity__TOU__usd'] - up01['cbill.electricity__TOU__usd'] 
up01['pct_saving_cbill.electricity_TOU_%'] = up01['saving_cbill.electricity_TOU_usd']/up01['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up01)

Package 2. Enhanced Enclosure - upgrade 2

In [9]:
up02_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['2'])]
up02 = results['2'].join(up02_baseline.set_index('building_id'), on='building_id')
up02 = up02.join(euss02.set_index('building_id'), on='building_id')
up02['saving_cbill.electricity_TOU_usd'] = up02['baseline_cbill.electricity__TOU__usd'] - up02['cbill.electricity__TOU__usd'] 
up02['pct_saving_cbill.electricity_TOU_%'] = up02['saving_cbill.electricity_TOU_usd']/up02['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up02)

Package 3. Mininum Efficiency Heat Pump with Electric Heat Backup - upgrade 3

In [10]:
up03_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['3'])]
up03 = results['3'].join(up03_baseline.set_index('building_id'), on='building_id')
up03 = up03.join(euss03.set_index('building_id'), on='building_id')
up03['saving_cbill.electricity_TOU_usd'] = up03['baseline_cbill.electricity__TOU__usd'] - up03['cbill.electricity__TOU__usd'] 
up03['pct_saving_cbill.electricity_TOU_%'] = up03['saving_cbill.electricity_TOU_usd']/up03['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up03)

Package 4. High Efficiency Heat Pump with Electric Heat Backup - upgrade 4

In [11]:
up04_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['4'])]
up04 = results['4'].join(up04_baseline.set_index('building_id'), on='building_id')
up04 = up04.join(euss04.set_index('building_id'), on='building_id')
up04['saving_cbill.electricity_TOU_usd'] = up04['baseline_cbill.electricity__TOU__usd'] - up04['cbill.electricity__TOU__usd'] 
up04['pct_saving_cbill.electricity_TOU_%'] = up04['saving_cbill.electricity_TOU_usd']/up04['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up04)

Package 5. Mininum Efficiency Heat Pump with Existing Heat Backup - upgrade 5

In [12]:
up05_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['5'])]
up05 = results['5'].join(up05_baseline.set_index('building_id'), on='building_id')
up05 = up05.join(euss05.set_index('building_id'), on='building_id')
up05['saving_cbill.electricity_TOU_usd'] = up05['baseline_cbill.electricity__TOU__usd'] - up05['cbill.electricity__TOU__usd'] 
up05['pct_saving_cbill.electricity_TOU_%'] = up05['saving_cbill.electricity_TOU_usd']/up05['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up05)

Package 6. Basic Enclosure + HPWH + High Efficiency HP/Electric Backup - upgrade 9 excluding: ["clothes_dryer", "range_oven"])

In [13]:
up06_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['6'])]
up06 = results['6'].join(up06_baseline.set_index('building_id'), on='building_id')
up06 = up06.join(euss09.set_index('building_id'), on='building_id')
up06['saving_cbill.electricity_TOU_usd'] = (up06['baseline_cbill.electricity__TOU__usd']-up06['baseline_cbill.electricity__clothes_dryer__TOU__usd'].fillna(0)-up06['baseline_cbill.electricity__range_oven__TOU__usd'].fillna(0)) - \
                                                      (up06['cbill.electricity__TOU__usd']-up06['cbill.electricity__clothes_dryer__TOU__usd'].fillna(0)-up06['cbill.electricity__range_oven__TOU__usd'].fillna(0)) 
up06['pct_saving_cbill.electricity_TOU_%'] = up06['saving_cbill.electricity_TOU_usd']/up06['baseline_cbill.electricity__TOU__usd'] * 100

Package 7. Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup - upgrade 10 excluding: ["clothes_dryer", "range_oven"])

In [14]:
up07_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['7'])]
up07 = results['7'].join(up07_baseline.set_index('building_id'), on='building_id')
up07 = up07.join(euss10.set_index('building_id'), on='building_id')
up07['saving_cbill.electricity_TOU_usd'] = (up07['baseline_cbill.electricity__TOU__usd']-up07['baseline_cbill.electricity__clothes_dryer__TOU__usd'].fillna(0)-up07['baseline_cbill.electricity__range_oven__TOU__usd'].fillna(0)) - \
                                                      (up07['cbill.electricity__TOU__usd']-up07['cbill.electricity__clothes_dryer__TOU__usd'].fillna(0)-up07['cbill.electricity__range_oven__TOU__usd'].fillna(0)) 
up07['pct_saving_cbill.electricity_TOU_%'] = up07['saving_cbill.electricity_TOU_usd']/up07['baseline_cbill.electricity__TOU__usd'] * 100


Package 8. Heat Pump Water Heater - upgrade 6

In [15]:
up08_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['8'])]
up08 = results['8'].join(up08_baseline.set_index('building_id'), on='building_id')
up08 = up08.join(euss06.set_index('building_id'), on='building_id')
up08['saving_cbill.electricity_TOU_usd'] = up08['baseline_cbill.electricity__TOU__usd'] - up08['cbill.electricity__TOU__usd'] 
up08['pct_saving_cbill.electricity_TOU_%'] = up08['saving_cbill.electricity_TOU_usd']/up08['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up08)

Package 9. Electric Clothes Dryer - Clothes dryer in upgrade 7

In [16]:
up09_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['9'])]
up09 = results['9'].join(up09_baseline.set_index('building_id'), on='building_id')
euss07_dryer = euss07.loc[euss07['building_id'].isin(building_ids_list['9'])]
up09 = up09.join(euss07_dryer.set_index('building_id'), on='building_id')
up09['saving_cbill.electricity_TOU_usd'] = up09['baseline_cbill.electricity__clothes_dryer__TOU__usd'].fillna(0) - up09['cbill.electricity__clothes_dryer__TOU__usd'].fillna(0) 
up09['pct_saving_cbill.electricity_TOU_%'] = up09['saving_cbill.electricity_TOU_usd']/up09['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up09)

Package 10. Heat Pump Clothes Dryer - Clothes dryer in upgrade 8

In [17]:
up10_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['10'])]
up10 = results['10'].join(up10_baseline.set_index('building_id'), on='building_id')
euss08_dryer = euss08.loc[euss08['building_id'].isin(building_ids_list['10'])]
up10 = up10.join(euss08_dryer.set_index('building_id'), on='building_id')
up10['saving_cbill.electricity_TOU_usd'] = up10['baseline_cbill.electricity__clothes_dryer__TOU__usd'].fillna(0) - up10['cbill.electricity__clothes_dryer__TOU__usd'].fillna(0) 
up10['pct_saving_cbill.electricity_TOU_%'] = up10['saving_cbill.electricity_TOU_usd']/up10['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up10)

Package 11. Electric Cooking - Cooking ("range_oven") in upgrade 7

In [18]:
up11_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['11'])]
up11 = results['11'].join(up11_baseline.set_index('building_id'), on='building_id')
euss07_cooking = euss07.loc[euss07['building_id'].isin(building_ids_list['11'])]
up11 = up11.join(euss07_cooking.set_index('building_id'), on='building_id')
up11['saving_cbill.electricity_TOU_usd'] = up11['baseline_cbill.electricity__clothes_dryer__TOU__usd'].fillna(0) - up11['cbill.electricity__clothes_dryer__TOU__usd'].fillna(0) 
up11['pct_saving_cbill.electricity_TOU_%'] = up11['saving_cbill.electricity_TOU_usd']/up11['baseline_cbill.electricity__TOU__usd'] * 100

Package 12. Induction Cooking - Cooking ("range_oven") in upgrade 8

In [19]:
up12_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['12'])]
up12 = results['12'].join(up12_baseline.set_index('building_id'), on='building_id')
euss08_cooking = euss08.loc[euss08['building_id'].isin(building_ids_list['12'])]
up12 = up12.join(euss08_cooking.set_index('building_id'), on='building_id')
up12['saving_cbill.electricity_TOU_usd'] = up12['baseline_cbill.electricity__clothes_dryer__TOU__usd'].fillna(0) - up12['cbill.electricity__clothes_dryer__TOU__usd'].fillna(0) 
up12['pct_saving_cbill.electricity_TOU_%'] = up12['saving_cbill.electricity_TOU_usd']/up12['baseline_cbill.electricity__TOU__usd'] * 100

Package 13. Mininum Efficiency Whole Home Electrification - upgrade 7

In [20]:
up13_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['13'])]
up13 = results['13'].join(up13_baseline.set_index('building_id'), on='building_id')
up13 = up13.join(euss07.set_index('building_id'), on='building_id')
up13['saving_cbill.electricity_TOU_usd'] = up13['baseline_cbill.electricity__TOU__usd'] - up13['cbill.electricity__TOU__usd'] 
up13['pct_saving_cbill.electricity_TOU_%'] = up13['saving_cbill.electricity_TOU_usd']/up13['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up13)

Package 14. high Efficiency Whole Home Electrification - upgrade 8

In [21]:
up14_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['14'])]
up14 = results['14'].join(up14_baseline.set_index('building_id'), on='building_id')
up14 = up14.join(euss08.set_index('building_id'), on='building_id')
up14['saving_cbill.electricity_TOU_usd'] = up14['baseline_cbill.electricity__TOU__usd'] - up14['cbill.electricity__TOU__usd'] 
up14['pct_saving_cbill.electricity_TOU_%'] = up14['saving_cbill.electricity_TOU_usd']/up14['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up14)

Package 15. Basic Enclosure + High Efficiency Whole Home Electrification - upgrade 9

In [22]:
up15_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['15'])]
up15 = results['15'].join(up15_baseline.set_index('building_id'), on='building_id')
up15 = up15.join(euss09.set_index('building_id'), on='building_id')
up15['saving_cbill.electricity_TOU_usd'] = up15['baseline_cbill.electricity__TOU__usd'] - up15['cbill.electricity__TOU__usd'] 
up15['pct_saving_cbill.electricity_TOU_%'] = up15['saving_cbill.electricity_TOU_usd']/up15['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up15)

Package 16. Enhanced Enclosure + High Efficiency Whole Home Electrification - upgrade 10

In [23]:
up16_baseline = euss00.loc[euss00['building_id'].isin(building_ids_list['16'])]
up16 = results['16'].join(up16_baseline.set_index('building_id'), on='building_id')
up16 = up16.join(euss10.set_index('building_id'), on='building_id')
up16['saving_cbill.electricity_TOU_usd'] = up16['baseline_cbill.electricity__TOU__usd'] - up16['cbill.electricity__TOU__usd'] 
up16['pct_saving_cbill.electricity_TOU_%'] = up16['saving_cbill.electricity_TOU_usd']/up16['baseline_cbill.electricity__TOU__usd'] * 100
all_frame.append(up16)

In [24]:
community_results_bill = pd.concat(all_frame)
community_results_bill = community_results_bill.drop(['cbill.electricity__TOU__usd',
                             'cbill.electricity__clothes_dryer__TOU__usd',
                             'cbill.electricity__range_oven__TOU__usd',
                             'baseline_cbill.electricity__clothes_dryer__TOU__usd',
                             'baseline_cbill.electricity__range_oven__TOU__usd'], axis=1)
community_results_bill['baseline_cbill.total_usd'] = community_results_bill['baseline_cbill.electricity__TOU__usd']+community_results_bill['baseline_cbill.natural_gas_usd']+community_results_bill['baseline_cbill.fuel_oil_usd']+community_results_bill['baseline_cbill.propane_usd']
community_results_bill['saving_cbill.total_usd'] = community_results_bill['saving_cbill.electricity_TOU_usd']+community_results_bill['saving_cbill.natural_gas_usd']+community_results_bill['saving_cbill.fuel_oil_usd']+community_results_bill['saving_cbill.propane_usd']
community_results_bill['pct_saving_cbill.total_%'] = community_results_bill['saving_cbill.total__TOU_usd']/community_results_bill['baseline_cbill.total__TOU_usd'] * 100

In [25]:
community_results_bill.to_csv(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results_TOU.csv")
community_results_bill.to_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results_TOU.parquet")